In [115]:
%matplotlib inline

import nltk
import re
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, Phrases, phrases

import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# Spacy for lemmatization
import spacy
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups

In [34]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Import the Newsgroups Data

In [32]:
newsgroups_train = fetch_20newsgroups(subset='train')

## Convert the newsgroups into Pandas DataFrame

In [30]:
data = zip(newsgroups_train.data, 
           newsgroups_train.target, 
           newsgroups_train.target_names)
df = pd.DataFrame(data=data, columns=['data', 'target', 'target_names'])
df.head()

,data,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,alt.atheism
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.graphics
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.os.ms-windows.misc
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.sys.ibm.pc.hardware
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,comp.sys.mac.hardware


In [33]:
df.target_names.unique()

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype=object)

## Cleansing of data

In [69]:
# Convert the data to list for preprocessing.
data = df.data.values.tolist()

# Remove emails.
data = [re.sub('\S*@\S*\s', '', sentence)
        for sentence in data]

# Remove new line characters.
data = [re.sub('\s', ' ', sentence)
        for sentence in data]

# Remove distracting single quotes.
data = [re.sub("\'", '', sentence)
        for sentence in data]

data = [sentence.strip()
        for sentence in data]

data[:2]

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15   I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is  all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail.  Thanks, - IL    ---- brought to you by your neighborhood Lerxst ----',
 'From: (Guy Kuo) Subject: SI Clock Poll - Final Call Summary: Final call for SI clock reports Keywords: SI,acceleration,clock,upgrade Article-I.D.: shelley.1qvfo9INNc3s Organization: University of Washington Lines: 11 NNTP-Posting-Host: carson.u.washington.edu  A fair number of brave souls who upg

## Tokenize sentence and clean up text.

In [74]:
def sentence_to_tokens(sentences):
    for sentence in sentences:
        yield simple_preprocess(str(sentence),
                                deacc=True) # Removes punctuation.

In [75]:
data_words = list(sentence_to_tokens(data))
data_words[:1]

[['from',
  'wheres',
  'my',
  'thing',
  'subject',
  'what',
  'car',
  'is',
  'this',
  'nntp',
  'posting',
  'host',
  'rac',
  'wam',
  'umd',
  'edu',
  'organization',
  'university',
  'of',
  'maryland',
  'college',
  'park',
  'lines',
  'was',
  'wondering',
  'if',
  'anyone',
  'out',
  'there',
  'could',
  'enlighten',
  'me',
  'on',
  'this',
  'car',
  'saw',
  'the',
  'other',
  'day',
  'it',
  'was',
  'door',
  'sports',
  'car',
  'looked',
  'to',
  'be',
  'from',
  'the',
  'late',
  'early',
  'it',
  'was',
  'called',
  'bricklin',
  'the',
  'doors',
  'were',
  'really',
  'small',
  'in',
  'addition',
  'the',
  'front',
  'bumper',
  'was',
  'separate',
  'from',
  'the',
  'rest',
  'of',
  'the',
  'body',
  'this',
  'is',
  'all',
  'know',
  'if',
  'anyone',
  'can',
  'tellme',
  'model',
  'name',
  'engine',
  'specs',
  'years',
  'of',
  'production',
  'where',
  'this',
  'car',
  'is',
  'made',
  'history',
  'or',
  'whatever',
  

## Creating bigram and trigram models.

In [77]:
bigram = Phrases(data_words, min_count=5, threshold=100)
trigram = Phrases(bigram[data_words], threshold=100)

In [85]:
bigram_model = phrases.Phraser(bigram)
trigram_model = phrases.Phraser(trigram)

In [89]:
trigram_model[bigram_model[data_words[0]]]

['from',
 'guy',
 'kuo',
 'subject',
 'si',
 'clock',
 'poll',
 'final',
 'call',
 'summary',
 'final',
 'call',
 'for',
 'si',
 'clock',
 'reports',
 'keywords',
 'si',
 'acceleration',
 'clock',
 'upgrade',
 'article',
 'shelley',
 'qvfo',
 'innc',
 'organization',
 'university',
 'of',
 'washington',
 'lines',
 'nntp_posting_host',
 'carson',
 'washington',
 'edu',
 'fair',
 'number',
 'of',
 'brave',
 'souls',
 'who',
 'upgraded',
 'their',
 'si',
 'clock',
 'oscillator',
 'have',
 'shared',
 'their',
 'experiences',
 'for',
 'this',
 'poll',
 'please',
 'send',
 'brief',
 'message',
 'detailing',
 'your',
 'experiences',
 'with',
 'the',
 'procedure',
 'top',
 'speed',
 'attained',
 'cpu',
 'rated',
 'speed',
 'add',
 'on',
 'cards',
 'and',
 'adapters',
 'heat',
 'sinks',
 'hour',
 'of',
 'usage',
 'per',
 'day',
 'floppy',
 'disk',
 'functionality',
 'with',
 'and',
 'floppies',
 'are',
 'especially',
 'requested',
 'will',
 'be',
 'summarizing',
 'in',
 'the',
 'next',
 'two',


## Remove stopwords, make bigrams/trigrams and lemmatize

In [95]:
def remove_stopwords(docs):
    return [[token for token in tokens
             if token not in stop_words]
            for tokens in docs]

def make_bigrams(docs):
    return [bigram_model[doc] for doc in docs]

def make_trigrams(docs):
    return [trigram_model[doc] for doc in docs]

# Initialize spacy 'en' model, keeping only tagget component
# for efficiency.
# !python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    result = []
    for tokens in docs:
        doc = nlp(' '.join(tokens))
        result.append([token.lemma_ for token in doc 
                       if token.pos_ in allowed_postags])
    return result

In [100]:
# Remove stop words.
data_words_nostops = remove_stopwords(data_words)

# Form bigrams.
data_words_bigrams = make_bigrams(data_words_nostops)

data_lemmatized = lemmatization(data_words_bigrams)

data_lemmatized[:1]

[['where',
  's',
  'thing',
  'car',
  'nntp_poste',
  'host',
  'rac',
  'wam',
  'umd',
  'organization',
  'university',
  'maryland',
  'college',
  'park',
  'line',
  'wonder',
  'anyone',
  'could',
  'enlighten',
  'car',
  'see',
  'day',
  'door',
  'sport',
  'car',
  'look',
  'late',
  'early',
  'call',
  'bricklin',
  'door',
  'really',
  'small',
  'addition',
  'front',
  'bumper',
  'separate',
  'rest',
  'body',
  'know',
  'anyone',
  'tellme',
  'model',
  'name',
  'engine',
  'specs',
  'year',
  'production',
  'car',
  'make',
  'history',
  'info',
  'funky',
  'look',
  'car',
  'mail',
  'thank',
  'bring',
  'neighborhood',
  'lerxst']]

## Create the dictionary and corpus needed for topic modelling

In [103]:
# Create dictionary.
id2word = corpora.Dictionary(data_lemmatized)

# Create corpus.
corpus = [id2word.doc2bow(text) for text in data_lemmatized]

# Gensim creates a unique id for each word in the document. 
# The result is a list of tuples (word_id, word_frequency).
corpus[:1]

[[(0, 1),
  (1, 2),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 5),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 2),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1)]]

In [104]:
# The word for the id = 0.
id2word[0]

'addition'

In [105]:
# Human readable format of the corpus.
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('bumper', 1),
  ('call', 1),
  ('car', 5),
  ('college', 1),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('specs', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('wam', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## Building the topic model.

In [107]:
# Build LDA model.
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [108]:
lda_model.print_topics()

[(0,
  '0.001*"access" + 0.001*"boot" + 0.001*"question" + 0.001*"get" + 0.001*"terminal" + 0.001*"network" + 0.001*"file" + 0.001*"well" + 0.001*"great" + 0.001*"have"'),
 (1,
  '0.001*"year" + 0.001*"car" + 0.001*"insurance" + 0.001*"rate" + 0.001*"go" + 0.001*"be" + 0.001*"high" + 0.001*"pay" + 0.001*"get" + 0.001*"live"'),
 (2,
  '0.014*"record" + 0.013*"old" + 0.013*"jame" + 0.013*"initially" + 0.013*"pretty" + 0.013*"much" + 0.012*"different" + 0.012*"try" + 0.012*"require" + 0.012*"absolute"'),
 (3,
  '0.001*"weapon" + 0.001*"year" + 0.001*"scsi" + 0.001*"not" + 0.001*"mass_destruction" + 0.001*"say" + 0.001*"would" + 0.001*"article" + 0.001*"car" + 0.001*"insurance"'),
 (4,
  '0.052*"board" + 0.016*"be" + 0.016*"file" + 0.016*"stac" + 0.016*"have" + 0.016*"autodoubler" + 0.016*"problem" + 0.011*"double" + 0.011*"illinoi" + 0.011*"write"'),
 (5,
  '0.026*"clock" + 0.021*"si" + 0.016*"cerkoney" + 0.016*"rod" + 0.016*"upgrade" + 0.016*"poll" + 0.016*"co" + 0.016*"experience" + 0.0

In [110]:
doc_lda = lda_model[corpus]

## Compute model perplexity and coherence score.

In [113]:
# A measure of how good the model is. The lower the better.
lda_model.log_perplexity(corpus)

-7.2952888443039825

In [114]:
# Compute coherence score.
coherence_model_lds = CoherenceModel(model=lda_model,
                                     texts=data_lemmatized,
                                     dictionary=id2word,
                                     coherence='c_v')
coherence_model_lds.get_coherence()

0.487836108430972

## Visualize the topics-keywords

In [116]:
pyLDAvis.enable_notebook()

In [117]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.227019  0.067517       1        1  27.593241
16     0.190079  0.151791       2        1  21.953465
10     0.065776 -0.221200       3        1   9.958546
17    -0.046169 -0.091857       4        1   7.673302
9      0.039464  0.014816       5        1   5.825302
19    -0.091185  0.057918       6        1   5.270487
14     0.014621 -0.079041       7        1   4.689174
11    -0.042214 -0.010801       8        1   3.924805
5      0.009567 -0.001253       9        1   3.797587
4      0.016664  0.055421      10        1   3.707621
18     0.049997  0.016473      11        1   2.637581
13    -0.011127 -0.010224      12        1   2.483528
2     -0.002067 -0.013463      13        1   0.382394
3      0.003118  0.009554      14        1   0.015871
0      0.005526  0.006996      15        1   0.015088
7      0.005393  0.006865      16        1   0.014794
15     0.005870  0.011230      17        1   0.014494
8      0.005568  0.010550      18        1   0.014425
1      0.002668  0.010328      19        1   0.014372
6      0.005470  0.008381      20        1   0.013930, topic_info=     Category       Freq        Term      Total  loglift  logprob
499   Default  28.000000        scsi  28.000000  30.0000  30.0000
402   Default   9.000000      weapon   9.000000  29.0000  29.0000
892   Default  15.000000      option  15.000000  28.0000  28.0000
7     Default  19.000000         car  19.000000  27.0000  27.0000
52    Default  29.000000        year  29.000000  26.0000  26.0000
1243  Default  15.000000   insurance  15.000000  25.0000  25.0000
533   Default   7.000000       board   7.000000  24.0000  24.0000
92    Default  14.000000        rate  14.000000  23.0000  23.0000
727   Default   6.000000      record   6.000000  22.0000  22.0000
1135  Default  10.000000        tiff  10.000000  21.0000  21.0000
368   Default  12.000000       power  12.000000  20.0000  20.0000
160   Default  14.000000         may  14.000000  19.0000  19.0000
164   Default   8.000000        much   8.000000  18.0000  18.0000
934   Default   9.000000         ssf   9.000000  17.0000  17.0000
880   Default   9.000000      module   9.000000  16.0000  16.0000
706   Default   7.000000       moral   7.000000  15.0000  15.0000
714   Default   6.000000         old   6.000000  14.0000  14.0000
162   Default  11.000000          mb  11.000000  13.0000  13.0000
403   Default  18.000000       would  18.000000  12.0000  12.0000
116   Default  19.000000          be  19.000000  11.0000  11.0000
55    Default   9.000000         add   9.000000  10.0000  10.0000
831   Default   8.000000      flight   8.000000   9.0000   9.0000
794   Default   8.000000  capability   8.000000   8.0000   8.0000
142   Default  14.000000          go  14.000000   7.0000   7.0000
253   Default  12.000000       write  12.000000   6.0000   6.0000
57    Default  13.000000     article  13.000000   5.0000   5.0000
641   Default   6.000000       child   6.000000   4.0000   4.0000
717   Default   6.000000      parent   6.000000   3.0000   3.0000
263   Default   7.000000        code   7.000000   2.0000   2.0000
247   Default   8.000000      system   8.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
742   Topic20   0.000335       swear   4.529913  -0.6324  -7.1954
263   Topic20   0.000341        code   7.659555  -1.1416  -7.1793
622   Topic20   0.000331    absolute   3.834465  -0.4785  -7.2082
706   Topic20   0.000340       moral   7.813648  -1.1640  -7.1819
707   Topic20   0.000331    morality   4.527295  -0.6434  -7.2069
709   Topic20   0.000325    multiple   2.891303  -0.2130  -7.2250
757   Topic20   0.000325        yhwh   2.886535  -0.2132  -7.2268
52    Topic20   0.000348        year  29.796482  -2.4792  -7.1585
690   Topic20   0.000329         jew   4.370025  -0.6166  -7.2156
1243  Topic20   0.000337   insurance  15.994181  -1.8897  -7.19

# References

- https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/